In [1]:
#library import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.stats import entropy
import math
from functools import reduce
from sklearn.feature_selection import mutual_info_classif


In [2]:
#data reading

data = pd.read_csv('GSE468.txt')
data.head()

,AFFX-MurIL2_at,AFFX-MurIL10_at,AFFX-MurIL4_at,AFFX-MurFAS_at,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,...,104_at,105_at,1216_at,788_s_at,426_at,397_at,132_at,hum_alu_at,class,sample
0,c,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,d,string
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class,meta
2,-222.867004,44.987,29.072001,107.036003,1633.719971,1792.088013,1450.660034,5234.332031,5408.90918,5754.067871,...,413.343994,54.146999,-43.158001,216.522003,-54.693001,-197.839005,94.212997,16497.83203,Met,GSM4312
3,-372.1480103,132.0480042,32.92200089,190.8150024,12774.59375,16425.99414,11523.09961,37526.45703,39596.67188,43983.26172,...,773.2199707,-47.15100098,52.86899948,-114.637001,-57.47499847,-123.4140015,309.8110046,87164.00781,Met,GSM4313
4,-64.248001,106.371002,7.49,284.80899,2461.489014,2229.948975,1757.072998,7717.64502,6697.812012,7588.408203,...,442.59201,162.167007,-7.781,304.131989,-62.653999,-354.554993,58.750999,16563.77734,Met,GSM4314


In [3]:
data = data.drop(data.index[[0,1]])
data.index=range(0,len(data),1)
data = data.drop(['sample'], axis=1)
data.dropna()
data.isnull().sum()

AFFX-MurIL2_at     0
AFFX-MurIL10_at    0
AFFX-MurIL4_at     0
AFFX-MurFAS_at     0
AFFX-BioB-5_at     0
                  ..
426_at             0
397_at             0
132_at             0
hum_alu_at         0
class              0
Length: 1466, dtype: int64

In [4]:
dict = {
    'class':{
        'Met':1,
        'NonMet':0,
    }
}   
data=data.replace(dict)
data.head()

,AFFX-MurIL2_at,AFFX-MurIL10_at,AFFX-MurIL4_at,AFFX-MurFAS_at,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,...,103_at,104_at,105_at,1216_at,788_s_at,426_at,397_at,132_at,hum_alu_at,class
0,-222.867004,44.987,29.072001,107.036003,1633.719971,1792.088013,1450.660034,5234.332031,5408.90918,5754.067871,...,160.125,413.343994,54.146999,-43.158001,216.522003,-54.693001,-197.839005,94.212997,16497.83203,1
1,-372.1480103,132.0480042,32.92200089,190.8150024,12774.59375,16425.99414,11523.09961,37526.45703,39596.67188,43983.26172,...,1926.875977,773.2199707,-47.15100098,52.86899948,-114.637001,-57.47499847,-123.4140015,309.8110046,87164.00781,1
2,-64.248001,106.371002,7.49,284.80899,2461.489014,2229.948975,1757.072998,7717.64502,6697.812012,7588.408203,...,162.720001,442.59201,162.167007,-7.781,304.131989,-62.653999,-354.554993,58.750999,16563.77734,1
3,-230.5480042,26.29800034,52.44900131,236.4019928,2040.295044,1866.427979,1657.119995,6508.688965,6581.226074,6658.895996,...,2664.638916,442.0679932,91.79199982,-3.720000029,182.1900024,-84.14199829,-153.6909943,96.18800354,6231.430176,1
4,270.537994,272.761993,-26.153999,207.901993,14976.5752,21094.98633,15805.98535,41854.03906,44083.20703,56576.92969,...,365.582001,703.106018,-42.016998,-147.490005,-168.417999,-213.264008,-200.826996,50.279999,81383.84375,1


In [5]:
#data split

X = data.drop(['class'], axis=1)
y = data['class']
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.6)
print(len(x_train),len(x_test),len(y_train),len(y_test))

9 14 9 14


In [6]:
#accuracies with classifier run WITHOUT feature selection

model1=SVC()
model2=GaussianNB()
model3 = DecisionTreeClassifier()
model4 = KNeighborsClassifier()
model1.fit(x_train, y_train)
print ('SVC Score:' ,model1.score(x_test, y_test))
model2.fit(x_train, y_train)
print ('NB Score:' ,model2.score(x_test, y_test))
model3.fit(x_train, y_train)
print ('DT Score:' ,model3.score(x_test, y_test))
model4.fit(x_train, y_train)
print ('KNN Score:' ,model4.score(x_test, y_test))

SVC Score: 0.5
NB Score: 0.5
DT Score: 0.2857142857142857
KNN Score: 0.42857142857142855


In [7]:
#FILTER TECHNIQUE 1

#correlation

df1 = X
op=y
a1=[]
c1=[]
for i in df1.columns.values:
        df1[i]=df1[i].astype('float64')
        correlation=op.corr(df1[i])
        if correlation>0.2:
            a1.append(correlation)
            c1.append(i)

dfc1=pd.DataFrame(c1)
dfa1=pd.DataFrame(a1)
info1 = pd.concat([dfc1,dfa1], axis=1)
info1.columns = ['name','correlation']
info1

,name,correlation
0,1009_at,0.209281
1,102_at,0.292888
2,109_at,0.274532
3,1131_at,0.238132
4,1137_at,0.392086
...,...,...
235,201_s_at,0.242146
236,148_at,0.311912
237,128_at,0.284704
238,129_g_at,0.243046


In [8]:
#selecting top 50 features

top1 = info1.nlargest(50,'correlation')
SelectedFeatures1= np.array(top1['name'])
SelectedFeatures1

array(['1818_at', '1104_s_at', '1771_s_at', '205_g_at', '671_at',
       '1054_at', '1693_s_at', '2062_at', '1470_at', '1007_s_at',
       '841_at', '1319_at', '1173_g_at', '829_s_at', '1237_at', '709_at',
       '1310_at', '1987_at', '941_at', '2031_s_at', '1840_g_at',
       '1718_at', '1637_at', '428_s_at', '892_at', '1741_s_at',
       '1830_s_at', '1199_at', '239_at', '1970_s_at', '591_s_at',
       '1993_s_at', '1563_s_at', '1825_at', '1137_at', '1242_at',
       '1143_s_at', '956_at', '1606_at', '981_at', '1837_at', '1363_at',
       '1884_s_at', '1451_s_at', '1449_at', '770_at', '2069_s_at',
       '1980_s_at', '1782_s_at', '1226_at'], dtype=object)

In [9]:
#FILTER TECHNIQUE 2

#info gain ratio

from scipy.stats import entropy
def info_gain(Ex,a):
    H_Ex = entropy(list(Counter(Ex).values()))
    # Compute the sum of all values v in a
    sum_v = 0
    for v in set(a):
        Ex_a_v = [x for x, t in zip(Ex, a) if t == v]
        sum_v += (len(Ex_a_v) / len(Ex)) *\
                 (entropy(list(Counter(Ex_a_v).values())))

    # Return result
    return H_Ex - sum_v


def intrinsic_value(Ex, a):
    # Compute the sum of all values v in a
    sum_v = 0
    for v in set(a):
        Ex_a_v = [x for x, t in zip(Ex, a) if t == v]
        sum_v += (len(Ex_a_v) / len(Ex)) * math.log(len(Ex_a_v) / len(Ex), 2)

    # Return result
    return -sum_v


def info_gain_ratio(Ex, a):
        # Check whether examples and attributes have the same lengths.
    if len(Ex) != len(a):
        raise ValueError("Ex and a must be of the same size.")

    # Compute information gain ratio as IG/IV
    return info_gain(Ex, a) / intrinsic_value(Ex, a)


In [10]:
ar=[]
br=[]
iv=[]
ig=[]
for (columnName, columnData) in X.iteritems():
        a=info_gain_ratio(columnData, y)
        c=intrinsic_value(columnData, y)
        d=info_gain(columnData, y)
        ar.append(a)
        br.append(columnName)
        iv.append(c)
        ig.append(d)
a1 = pd.DataFrame(ar)
b1 = pd.DataFrame(br)
info2 = pd.concat([a1,b1], axis=1)
info2.columns = ['Score','Features']
info2

,Score,Features
0,0.693147,AFFX-MurIL2_at
1,0.693147,AFFX-MurIL10_at
2,0.693147,AFFX-MurIL4_at
3,0.693147,AFFX-MurFAS_at
4,0.693147,AFFX-BioB-5_at
...,...,...
1460,0.693147,788_s_at
1461,0.693147,426_at
1462,0.693147,397_at
1463,0.693147,132_at


In [11]:
#selecting top 50 features

top2 = info2.nlargest(50,'Score')
SelectedFeatures2= np.array(top2['Features'])
SelectedFeatures2

array(['AFFX-MurIL2_at', 'AFFX-MurIL10_at', 'AFFX-MurIL4_at',
       'AFFX-MurFAS_at', 'AFFX-BioB-5_at', 'AFFX-BioB-M_at',
       'AFFX-BioB-3_at', 'AFFX-BioC-5_at', 'AFFX-BioC-3_at',
       'AFFX-BioDn-5_at', 'AFFX-BioDn-3_at', 'AFFX-CreX-5_at',
       'AFFX-CreX-3_at', 'AFFX-DapX-5_at', 'AFFX-DapX-M_at',
       'AFFX-LysX-3_at', 'AFFX-ThrX-M_at', 'AFFX-TrpnX-5_at',
       'AFFX-TrpnX-3_at', 'AFFX-HUMISGF3A/M97935_MA_at',
       'AFFX-HUMISGF3A/M97935_MB_at', 'AFFX-HUMISGF3A/M97935_3_at',
       'AFFX-HUMRGE/M10098_5_at', 'AFFX-HUMRGE/M10098_M_at',
       'AFFX-HUMRGE/M10098_3_at', 'AFFX-HUMGAPDH/M33197_5_at',
       'AFFX-HUMGAPDH/M33197_M_at', 'AFFX-HUMGAPDH/M33197_3_at',
       'AFFX-HSAC07/X00351_5_at', 'AFFX-HSAC07/X00351_M_at',
       'AFFX-HSAC07/X00351_3_at', 'AFFX-HUMTFRR/M11507_5_at',
       'AFFX-HUMTFRR/M11507_3_at', 'AFFX-M27830_5_at', 'AFFX-M27830_M_at',
       'AFFX-YEL002c/WBP1_at', 'AFFX-YEL024w/RIP1_at',
       'AFFX-YEL021w/URA3_at', '1001_at', '1009_at', '102_at',


In [12]:
#FILTER TECHNIQUE 3

#Symmetrical Uncertainty


#entropy
from scipy.stats import entropy

#Joint Entropy
def jEntropy(Y,X):
    YX = np.c_[Y,X]
    return entropy(YX)

#conditional entropy
def cEntropy(Y, X):
    return jEntropy(Y, X) - entropy(X)

#SU
def symmetricalUncertain(Y,X):
    n = float(y.shape[0])
    vals = np.unique(Y)
    # Computing Entropy for the feature x. 
    Hx = entropy(X)
    # Computing Entropy for the feature y.
    Hy = entropy(Y)
    #Computing Joint entropy between x and y.
    Hxy = jEntropy(Y,X)
    IG = Hx-Hxy
    return 2*IG/(Hx+Hy)

In [13]:
top_n1 =[]
for i in X.columns.values:
    X[i]=X[i].astype('float64')
    top_n1.append(symmetricalUncertain(X[i],y))
#print (top_n1)
top_n2=[row[0] for row in top_n1]
print (top_n2)

col_name1 = np.array(X.columns)
a1 = pd.DataFrame(top_n2)
b1 = pd.DataFrame(col_name1)
info3 = pd.concat([a1,b1], axis=1)
info3.columns = ['Score','Features']
info3


<ipython-input-12-677bcc6d6073>:29: RuntimeWarning: invalid value encountered in true_divide
  return 2*IG/(Hx+Hy)


[nan, nan, nan, -0.13562750897618991, nan, nan, nan, -0.12960469055814935, nan, nan, -0.14948926925718456, nan, -0.185660415119528, nan, nan, nan, nan, nan, nan, nan, -0.1747756512400126, -0.2724711305450999, nan, nan, nan, -0.2890515093403299, -0.2707902754031973, -0.2788633989867532, -0.27582679646766844, -0.280113411725839, -0.2724981643215016, nan, nan, -0.11164286967178957, -0.1483178770103943, nan, 0.3155284210251379, -0.17278146991224266, -0.14350175193474551, -0.27839554333010963, nan, -0.2787255623349207, -0.17625828840244026, -0.2611291137798154, nan, nan, -0.24231519478860822, nan, -0.2846594374635179, -0.2821549158895663, nan, -0.29288336682958194, nan, -0.2920381128079479, -0.2856176014136893, nan, -0.27122758667821006, -0.25771204401744935, nan, nan, -0.2388303032788123, -0.2896802217772582, nan, nan, -0.27652197177890486, nan, -0.19840690364225994, -0.29755703423176144, -0.28977344785801046, -0.30188565740225803, -0.2923353353652115, nan, -0.16472983617723488, nan, nan, 

,Score,Features
0,NaN,AFFX-MurIL2_at
1,NaN,AFFX-MurIL10_at
2,NaN,AFFX-MurIL4_at
3,-0.135628,AFFX-MurFAS_at
4,NaN,AFFX-BioB-5_at
...,...,...
1460,NaN,788_s_at
1461,NaN,426_at
1462,NaN,397_at
1463,NaN,132_at


In [14]:
#selecting top 50

top3 = info3.nlargest(50,'Score')
SelectedFeatures3 = np.array(top3['Features'])
SelectedFeatures3

array(['AFFX-YEL024w/RIP1_at', '291_s_at', '379_at', '415_at', '600_at',
       '554_at', '606_at', '1916_s_at', '362_at', '909_g_at', '191_at',
       '224_at', '1372_at', '377_g_at', '1190_at', '934_at', '394_at',
       '740_at', '1338_s_at', '422_s_at', '1894_f_at', '852_at',
       '795_s_at', '595_at', '359_at', '414_at', '2057_g_at', '828_at',
       '230_s_at', '248_at', '1083_s_at', 'hum_alu_at', '173_at',
       'AFFX-HSAC07/X00351_3_st', '812_at', '1196_at', 'AFFX-M27830_5_at',
       '330_s_at', '753_at', '455_at', '878_s_at', '1008_f_at',
       'AFFX-BioC-5_at', '1280_i_at', '649_s_at', '1110_at', '1481_at',
       'AFFX-MurFAS_at', '370_at', '1001_at'], dtype=object)

In [15]:
#Selecting 5k subset

def top_fs_union(n):
    f1 = SelectedFeatures1[:n]
    f2 = SelectedFeatures2[:n]
    f3 = SelectedFeatures3[:n]
    f_3k = reduce(np.union1d, (f1,f2,f3))
    f_2k = ((len(f_3k)*2)//3)
    f_2k_random = np.random.choice(f_3k,f_2k,replace=False)
    f_5k = reduce(np.union1d, (f_3k,f_2k_random))
    #print (f_5k)
    return(f_5k)

In [16]:
#Assemble Function RUN

def assemble_run(n,classifier,x_train, x_test, y_train, y_test):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['class'], axis=0)
  df = data.loc[:,top_fs]
  X = df.drop(['class'], axis=1)
  y = df['class']
  model =classifier()#modify the code ,keep classifier in different function and call here
  model.fit(x_train, y_train)
  print (model.score(x_test,y_test))

In [17]:
#accuracies with classifier run WITH feature selection

for i in (5,10,15,20,25,30,40,50):
    print('Score for SVM by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,SVC,x_train, x_test, y_train, y_test)
print('\n\n')

#NaiveBayes
for i in (5,10,15,20,25,30,40,50):
    print('Score for Naive Bayes by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,GaussianNB,x_train, x_test, y_train, y_test)
print('\n\n')

#DecisionTree
for i in (5,10,15,20,25,30,40,50):
    print('Score for DecisionTree by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,DecisionTreeClassifier,x_train, x_test, y_train, y_test)
print('\n\n')

#KNN
for i in (5,10,15,20,25,30,40,50):
    print('Score for KNN by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,KNeighborsClassifier,x_train, x_test, y_train, y_test)
print('\n\n')


Score for SVM by Union of 5 Best Features: 0.5
Score for SVM by Union of 10 Best Features: 0.5
Score for SVM by Union of 15 Best Features: 0.5
Score for SVM by Union of 20 Best Features: 0.5
Score for SVM by Union of 25 Best Features: 0.5
Score for SVM by Union of 30 Best Features: 0.5
Score for SVM by Union of 40 Best Features: 0.5
Score for SVM by Union of 50 Best Features: 0.5



Score for Naive Bayes by Union of 5 Best Features: 0.5
Score for Naive Bayes by Union of 10 Best Features: 0.5
Score for Naive Bayes by Union of 15 Best Features: 0.5
Score for Naive Bayes by Union of 20 Best Features: 0.5
Score for Naive Bayes by Union of 25 Best Features: 0.5
Score for Naive Bayes by Union of 30 Best Features: 0.5
Score for Naive Bayes by Union of 40 Best Features: 0.5
Score for Naive Bayes by Union of 50 Best Features: 0.5



Score for DecisionTree by Union of 5 Best Features: 0.5714285714285714
Score for DecisionTree by Union of 10 Best Features: 0.42857142857142855
Score for DecisionTr